In [188]:
%matplotlib inline
import tessitura_tools as tt
import importlib
from ipywidgets import widgets
from IPython.display import display, HTML, FileLink
from functools import partial
import warnings
from pandas import DataFrame, to_datetime, read_csv
from matplotlib.figure import Figure
from simplecrypt import decrypt # package is called simple-crypt
from io import StringIO
from urllib.request import urlretrieve
import seaborn as sb
warnings.filterwarnings('ignore')

importlib.reload(tt);

In [189]:
interface_built = False

def decrypt_file(*args):
    
    # This function reads the inputted passwords and uses it to try and decrypt
    # the data file. A failed password raises an exception which is used to 
    # reset the password field with an error message.
    
    global data
    pw_status.value = 'Loading...'
    file = open('file.enc','rb').read()
    try:
        CSVplaintext = decrypt(pw_field.value, file).decode('utf8')
        file_like = StringIO(CSVplaintext)
        data = read_csv(file_like).drop('Unnamed: 0', axis=1) 
        data['Perf date'] = to_datetime(data['Perf date'])
        data['Order date'] = to_datetime(data['Order date'])
        updated_str = str(max(data['Order date']))[0:10]
        file_date_status.value = 'Updated through ' + updated_str
        pw_field.value = ''
        login_output.clear_output()
        if not interface_built:
            show_interface()
    except:
        pw_field.value = ''
        pw_status.value = 'Password is not correct'
    

pw_field = widgets.Password(layout=widgets.Layout(width='18%'))
pw_field.on_submit(decrypt_file)
pw_submit_button = widgets.Button(description='Submit',layout=widgets.Layout(width='8%'))
pw_submit_button.on_click(decrypt_file)
pw_status = widgets.Label('')
login_output = widgets.Output()
display(login_output)
# Download the latest data file from Dropbox
with login_output:
    display(HTML('<h1>FWMSH QuickData</h1>'))
    display(HTML('Loading data...'))
url = 'https://dl.dropboxusercontent.com/s/mwqvbwadqnys8eh/museum_data.enc?dl=0'
urlretrieve(url, './file.enc')

# Once loaded, show the password prompt
login_output.clear_output(wait=True)
with login_output:
    display(HTML('<h1>FWMSH QuickData</h1>'))    
    display(widgets.HBox([widgets.Label('Password:'),pw_field,pw_submit_button, pw_status]))

Output()

In [190]:
output = widgets.Output(layout={'border': '1px solid black'})
output

Output(layout=Layout(border='1px solid black'))

In [191]:
#data = tt.load_data('daily_charts')

# The context holds a reference to the current result in the
# output window
context = None
sort_column = 'Perf date'
sort_ascending = False

def do_search(*args):
    
    global context
    global sort_column
    global sort_ascending
    output.clear_output()        
    
    # Select venue
    venue_option = ''
    if venue_field.value != 'All':
        venue_option = venue_field.value
        
    # Select audience
    audience_option = ''
    if audience_field.value != 'All':
        audience_option = audience_field.value
        
    # Day of week
    weekday = False
    weekend = False
    day_of_week = -1
    if weekday_field.value == 'Weekdays':
        weekday=True
    elif weekday_field.value == 'Weekends':
        weekend=True
    elif weekday_field.value == 'Monday':
        day_of_week = 0
    elif weekday_field.value == 'Tuesday':
        day_of_week = 1
    elif weekday_field.value == 'Wednesday':
        day_of_week = 2
    elif weekday_field.value == 'Thursday':
        day_of_week = 3
    elif weekday_field.value == 'Friday':
        day_of_week = 4
    elif weekday_field.value == 'Saturday':
        day_of_week = 5
    elif weekday_field.value == 'Sunday':
        day_of_week = 6
    
    
    # Group by
    group = ''
    if group_field.value == 'Show':
        group = 'show'
    elif group_field.value == 'Day':
        group = 'd'
    elif group_field.value == 'Week':
        group = 'w'
    elif group_field.value == 'Month':
        group = 'm'
    elif group_field.value == 'Calendar Year':
        group = 'Y'
    elif group_field.value == 'Fiscal Year':
        group = 'A-SEP'
    
    try:
        result = data.tt.search(name=name_field.value,
                               date=[min_date_field.value, max_date_field.value],
                               time=time_field.value,
                               audience=audience_option,
                               group=group,
                               venue=venue_option,
                               day_of_week=day_of_week,
                               weekday=weekday,
                               weekend=weekend)
        #if (sort_field.value == 'Newest first') and ('Perf date' in result):
        #    result = result.sort_values('Perf date', ascending=False)
        #elif (sort_field.value == 'Oldest first') and ('Perf date' in result):
        #    result = result.sort_values('Perf date', ascending=True)
        #elif sort_field.value == 'A-Z':
        #    result = result.sort_values('Description', ascending=True)
        #elif sort_field.value == 'Z-A':
        #    result = result.sort_values('Description', ascending=False)
        if sort_column in result:
            result = result.sort_values(sort_column, ascending=sort_ascending)
        context = result       
    except Exception as e:
        #print(e)
        result = DataFrame()
        with output:
            display(HTML('Search error: a common error is bad formatting for the time field.'))
            
    with output:        
        # Print the number of results
        display(HTML(str(len(result)) + ' results'))

        # Show the contexual buttons if conditions are correct.
        if len(result) > 0:
            add_context_buttons()
            add_sort_buttons()
            with output:
                display(result)
    
    return(result)
    
def reset_search(*args):
    
    # Function to clear the search fields and the output
    
    global sort_column
    global sort_ascending
    
    sort_column = 'Perf date'
    sort_ascending = False
    
    name_field.value = ''
    min_date_field.value = None
    max_date_field.value = None
    time_field.value = ''
    venue_field.value = 'All'
    audience_field.value = 'All'
    weekday_field.value = 'All'
    group_field.value = 'None'
    sort_field.value = 'Newest first'
    output.clear_output()
    
def add_sort_buttons(*args):
    
    # Function to figure out which columns are sortable and create buttons for them    
    # We have a DataFrame
    
    global sort_column
    
    if type(context) == DataFrame:
        button_bar = widgets.HBox(width='auto')
        button_bar.children += (widgets.Label(value='Sort by:'),)
        for key in context:
            if len(context[key].unique()) > 1:
                if key == sort_column:
                    icon = 'check'
                else:
                    icon = ''
                but = widgets.Button(description=key, icon=icon, layout=widgets.Layout(width='100px'))
                but.on_click(partial(update_sort,key))
                button_bar.children += (but,)
                
        # Display the buttons
        with output:
            display(button_bar)        

def update_sort(col, *args):
    
    # Called when a sort button is pressed to change the sort behavior.
    
    global sort_column
    global sort_ascending
    global context
    
    if sort_column == col:
        sort_ascending = not sort_ascending # Flip between True and False
    else:
        sort_column = col
        
    if sort_column in context:
        context = context.sort_values(sort_column, ascending=sort_ascending)
        output.clear_output(wait=True)
        with output:
            add_context_buttons()
            add_sort_buttons()
            display(context)


def add_context_buttons(*args):
    
    # Create a bar to hold the buttons
    button_bar = widgets.HBox(width='auto')
    
    # We have a DataFrame
    if type(context) == DataFrame:
        if (group_field.value == 'None') and ('Description' in context):
            button_bar.children += (sales_plot_button,)
        elif (group_field.value in ['Day', 'Month']) and (len(context) > 1):
            button_bar.children += (revenue_plot_button, tickets_plot_button)
        save_button.description = 'Save as Excel'
        button_bar.children += (save_button,)
        if len(context) > 1:
            button_bar.children += (sum_button, mean_button)
            
    # We have a plot
    if type(context) == Figure:        
        save_button.description = 'Save Graph'
        button_bar.children += (save_button,)
                
    # Display the buttons
    with output:
        display(button_bar)        
    
def make_revenue_plot(*args):
    
    global context
    
    tt.set_plot_size(16,10)
    
    if type(context) == DataFrame:
        output.clear_output()
        plot = context.tt.create_revenue_chart()
        context = plot
        add_context_buttons()
        with output:
            display(plot)
        
def make_tickets_plot(*args):
    
    global context
    
    tt.set_plot_size(16,10)
    
    if type(context) == DataFrame:
        output.clear_output()
        plot = context.tt.create_tickets_chart()
        context = plot
        add_context_buttons()
        with output:
            display(plot)

def make_sales_plot(*args):
    
    global context
    
    tt.set_plot_size(16,10)

    if type(context) == DataFrame:
        output.clear_output()
        if len(context) < 10:
            plot = data.tt.create_sales_chart(context, end_on_event=True)
            context = plot
            add_context_buttons()
            with output:
                display(plot)
        else:
            with output:
                print('Error: Cannot create a sales graph for more than ten results. You have ' + str(len(context))+'.')

def master_save(*args):
        
    # We have a DataFrame
    if type(context) == DataFrame:
        save_xlsx(*args)
    
    # We have a plot
    if type(context) == Figure:
        save_plot(*args)
        
def save_plot(*args):
    if type(context) == Figure:
        context.savefig('saved_graph.png', dpi=300, bbox_inches='tight')
        output.clear_output()
        with output:
            display(HTML('Click the following link to download the file:'))
            display(FileLink("saved_graph.png"))
            
def save_xlsx(*args):
    #result = do_search().reset_index()
    if type(context) == DataFrame:
        context.to_excel('saved_data.xlsx')
        output.clear_output()
        with output:
            display(HTML('Click the following link to download the file:'))
            display(FileLink("saved_data.xlsx"))

def choose_autocomplete(*args, value):
    
    # This is called by button.on_click when a user selects
    # an autocomplete choice
    
    name_field.value = value
    autocomplete_preview.clear_output()
    do_search()
            
def autocomplete(*args, select=False):
    
    # Function to populate a list of matching shows as a user
    # types in the name field. Setting select=True will set
    # the name field to the first selection in the list.
    
    try:
        partial_matches = data[data['Description'].str.lower().str.contains(name_field.value.lower())]['Description'].unique()
    except:
        partial_matches = list()
        
    if (select == False) and (name_field.value not in partial_matches):
        
        # Create a button for the first five partial matches
        if name_field.value != '':
            autocomplete_preview.clear_output(wait=True)
            autocomplete_bar = widgets.HBox(layout=widgets.Layout(width='100%',display='inline-flex',flex_flow='row wrap'))
            for item in partial_matches[0:min(len(partial_matches), 5)]:
                temp_button = widgets.Button(description=item, layout=widgets.Layout(width='18%'))
                temp_button.on_click(partial(choose_autocomplete, value=item))
                autocomplete_bar.children += (temp_button,)

            with autocomplete_preview:
                display(autocomplete_bar)
        else:
            autocomplete_preview.clear_output()


    elif (select == True) and (len(name_field.value) > 0) and (len(partial_matches) > 0):
        autocomplete_preview.clear_output()
        name_field.value = partial_matches[0]
        do_search()
    else:
        autocomplete_preview.clear_output()

def data_sum(*args):
    global context
    if type(context) == DataFrame:       
        context = DataFrame(context.sum()).T[['Tickets', 'Revenue']]
        output.clear_output()
        add_context_buttons()
        with output:
            display(context)
            
def data_mean(*args):
    global context
    if type(context) == DataFrame:       
        context = DataFrame(context.mean()).T
        output.clear_output()
        add_context_buttons()
        with output:
            display(context)

In [192]:
file_date_status = widgets.Label('')

name_field = widgets.Text(description='Name', layout=widgets.Layout(width='30%'))
name_field.observe(autocomplete, names='value')
name_field.on_submit(partial(autocomplete, select=True))
autocomplete_preview = widgets.Output(layout={'border': '1px solid black'})

min_date_field = widgets.DatePicker(description='Start date', layout=widgets.Layout(width='24%'))
max_date_field = widgets.DatePicker(description='End date', layout=widgets.Layout(width='24%'))
time_field = widgets.Text(description='Time', layout=widgets.Layout(width='19%'))
time_field.on_submit(do_search)
basic_box = widgets.HBox([name_field, min_date_field, max_date_field, time_field],
                           layout=widgets.Layout(width='100%',display='inline-flex',flex_flow='row wrap'))


venue_field = widgets.Dropdown(description='Venue',
                                    layout=widgets.Layout(width='200px'),
                                    options=['All', 'Noble', 'Omni'],
                                    value='All')
venue_field.observe(do_search, names='value')
audience_field = widgets.Dropdown(description='Audience',
                                    layout=widgets.Layout(width='200px'),
                                    options=['All', 'Public', 'School'],
                                    value='All')
audience_field.observe(do_search, names='value')
weekday_field = widgets.Dropdown(description='Days of week',
                                    layout=widgets.Layout(width='200px'),
                                    options=['All', 'Weekdays', 'Weekends', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
                                    value='All')
weekday_field.observe(do_search, names='value')
group_field = widgets.Dropdown(description='Group by',
                                    layout=widgets.Layout(width='200px'),
                                    options=['None', 'Show', 'Day', 'Week', 'Month', 'Calendar Year', 'Fiscal Year'],
                                    value='None')
group_field.observe(do_search, names='value')
#sort_field = widgets.Dropdown(description='Sort by',
#                                    #layout={'border': '1px solid black'},
#                                    options=['Newest first', 'Oldest first', 'A-Z', 'Z-A'],
#                                    value='Newest first')
#sort_field.observe(do_search, names='value')

group_field.observe(do_search, names='value')
options_box = widgets.HBox([venue_field, audience_field, weekday_field, group_field],
                           layout=widgets.Layout(width='100%',display='inline-flex',flex_flow='row wrap'))

search_button = widgets.Button(description='Search')
search_button.on_click(do_search)
reset_button = widgets.Button(description='Clear')
reset_button.on_click(reset_search)
search_box = widgets.HBox([search_button, reset_button],
                           layout=widgets.Layout(width='100%',display='inline-flex',flex_flow='row wrap'))


# Contextually-aware buttons
sales_plot_button = widgets.Button(description='Make a sales graph', layout=widgets.Layout(width='175px'))
sales_plot_button.on_click(make_sales_plot)
revenue_plot_button = widgets.Button(description='Make a revenue graph', layout=widgets.Layout(width='175px'))
revenue_plot_button.on_click(make_revenue_plot)
tickets_plot_button = widgets.Button(description='Make a tickets graph', layout=widgets.Layout(width='175px'))
tickets_plot_button.on_click(make_tickets_plot)
save_button = widgets.Button(description='Save as Excel')
save_button.on_click(master_save)
sum_button = widgets.Button(description='Sum data')
sum_button.on_click(data_sum)
mean_button = widgets.Button(description='Average data')
mean_button.on_click(data_mean)


top_box = widgets.VBox([file_date_status, basic_box, autocomplete_preview, options_box, search_box])

def show_interface(*args):
    global interface_built
    display(top_box)
    interface_built = True

In [193]:
# Uncomment to hide code

#HTML('''<script>
#code_show=true; 
#function code_toggle() {
# if (code_show){
# $('div.input').hide();
# } else {
# $('div.input').show();
# }
# code_show = !code_show
#} 
#$( document ).ready(code_toggle);
#</script>
#<form action="javascript:code_toggle()"><input type="submit" value="Advanced: show code"></form>''')